In [ ]:
!pip install -Uq xgboost

# Amazon SageMaker XGBoost Bring Your Own Model
_**Hosting a Pre-Trained scikit-learn Model in Amazon SageMaker XGBoost Algorithm Container**_

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Optionally, train a scikit learn XGBoost model](#Optionally,-train-a-scikit-learn-XGBoost-model)
1. [Upload the pre-trained model to S3](#Upload-the-pre-trained-model-to-S3)
1. [Set up hosting for the model](#Set-up-hosting-for-the-model)
1. [Validate the model for use](#Validate-the-model-for-use)




---
## Background

Amazon SageMaker includes functionality to support a hosted notebook environment, distributed, serverless training, and real-time hosting. We think it works best when all three of these services are used together, but they can also be used independently.  Some use cases may only require hosting.  Maybe the model was trained prior to Amazon SageMaker existing, in a different service.

This notebook shows how to use a pre-existing scikit-learn trained XGBoost model with the Amazon SageMaker XGBoost Algorithm container to quickly create a hosted endpoint for that model. Please note that scikit-learn XGBoost model is compatible with SageMaker XGBoost container, whereas other gradient boosted tree models (such as one trained in SparkML) are not.

---
## Setup

Let's start by specifying:

* AWS region.
* The IAM role arn used to give learning and hosting access to your data. See the documentation for how to specify these.
* The S3 bucket that you want to use for training and model data.

In [3]:
%%time

import os
import boto3
import re
import json
import sagemaker
import datetime
from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

bucket = sagemaker.Session().default_bucket()

CPU times: user 768 ms, sys: 111 ms, total: 878 ms
Wall time: 1.77 s


In [5]:
now = datetime.datetime.now().strftime("%Y%m%d-%H-%M-%S")
prefix = "sagemaker/xgboost-byo-12" + now
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)
# customize to your bucket where you have stored the data

## Optionally, train a scikit learn XGBoost model

These steps are optional and are needed to generate the scikit-learn model that will eventually be hosted using the SageMaker Algorithm contained. 

### Install XGboost
Note that for conda based installation, you'll need to change the Notebook kernel to the environment with conda and Python3. 

In [6]:
!conda install -y -c conda-forge xgboost==0.90

Solving environment: done

# All requested packages already installed.



### Fetch the dataset

In [7]:
!aws s3 ls s3://sagemaker-sample-files/datasets/image/MNIST/

                           PRE MNIST/
                           PRE model/
                           PRE pytorch/
                           PRE test/
                           PRE train/
2020-11-18 02:38:22          0 
2020-12-28 19:34:02  220080426 mnist.pkl
2021-05-20 17:34:52   16168813 mnist.pkl.gz
2020-11-18 02:41:35    1648877 t10k-images-idx3-ubyte.gz
2020-11-18 02:41:27       4542 t10k-labels-idx1-ubyte.gz
2020-11-18 02:41:45    9912422 train-images-idx3-ubyte.gz
2020-11-18 02:41:28      28881 train-labels-idx1-ubyte.gz


### Prepare the dataset for training

In [8]:
%%time
import pickle
import boto3
import gzip

# Get the data from a public S3
buf=boto3.client('s3').get_object(
    Bucket='sagemaker-sample-files',
    Key='datasets/image/MNIST/mnist.pkl.gz'
)['Body'].read()


# decompress the buffer
decomp_buf = gzip.decompress(buf)
train_set, valid_set, test_set = pickle.loads(decomp_buf, encoding="latin1")

CPU times: user 1.03 s, sys: 418 ms, total: 1.45 s
Wall time: 2.88 s


In [9]:
train_X = train_set[0]
train_y = train_set[1]

valid_X = valid_set[0]
valid_y = valid_set[1]

test_X = test_set[0]
test_y = test_set[1]

### Train the XGBClassifier

In [10]:
import xgboost as xgb
import sklearn as sk

bt = xgb.XGBClassifier(
    max_depth=5, learning_rate=0.2, n_estimators=10, objective="multi:softmax"
)  # Setup xgboost model
bt.fit(train_X, train_y, eval_set=[(valid_X, valid_y)], verbose=False)  # Train it to our data

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

### Save the trained model file
Note that the model file name must satisfy the regular expression pattern: `^[a-zA-Z0-9](-*[a-zA-Z0-9])*;`. The model file also need to tar-zipped. 

In [11]:
model_file_name = "local-xgboost-model"+now
bt.save_model(model_file_name)

In [12]:
!tar czvf model.tar.gz $model_file_name

local-xgboost-model20211216-16-44-22


## Upload the pre-trained model to S3

In [13]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

## Set up hosting for the model

### Import model into hosting
This involves creating a SageMaker model from the model file previously uploaded to S3.

In [14]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-2")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [ ]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)
sm_client = boto3.client("sagemaker")

print(model_url)

In [ ]:
%%time
primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

# Register Model

In [17]:
model_package_group_name = 'Group'+now
config = {'ModelPackageGroupName': model_package_group_name, 
          'ModelPackageGroupDescription': 'XGboost model'
         }

### List existing model package groups

In [19]:
sm_client.list_model_package_groups(NameContains=model_package_group_name)['ModelPackageGroupSummaryList']
# sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)


[]

### Create a model package group

In [21]:
if not sm_client.list_model_package_groups(NameContains=model_package_group_name)['ModelPackageGroupSummaryList']:
    response = sm_client.create_model_package_group(**config)
else:
    print('Model Package Group already exists!')

### Optional: check to see if the model package group is createad

In [22]:
sm_client.list_model_package_groups(NameContains=model_package_group_name)['ModelPackageGroupSummaryList']
# sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)


[{'ModelPackageGroupName': 'Group20211216-16-44-22',
  'ModelPackageGroupArn': 'arn:aws:sagemaker:us-west-2:770499558699:model-package-group/group20211216-16-44-22',
  'ModelPackageGroupDescription': 'XGboost model',
  'CreationTime': datetime.datetime(2021, 12, 16, 16, 52, 30, 521000, tzinfo=tzlocal()),
  'ModelPackageGroupStatus': 'Completed'}]

In [23]:
modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": container,
         }
      ],
      "SupportedContentTypes": [ "text/csv" ],
      "SupportedResponseMIMETypes": [ "text/csv" ],
   }
 }

# Specify the model data
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]=model_url

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "Model to detect different types of ....",
    "ModelApprovalStatus" : "Approved"
}
create_model_package_input_dict.update(modelpackage_inference_specification)

In [24]:
create_mode_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:us-west-2:770499558699:model-package/group20211216-16-44-22/1


# Optional: Create endpoint configuration

SageMaker supports configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. In order to support this, you can create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way. In addition, the endpoint configuration describes the instance type required for model deployment.

### Create endpoint
Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes to complete.

## Validate the model for use
Now you can obtain the endpoint from the client library using the result from previous operations and generate classifications from the model using that endpoint.

Lets generate the prediction for a single datapoint. We'll pick one from the test data generated earlier.

### Post process the output
Since the result is a string, let's process it to determine the the output class label. 

### (Optional) Delete the Endpoint

If you're ready to be done with this notebook, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.